In [1]:
import numpy as np
import os
import sys
import re
from functools import reduce

In [40]:
query_file = 'queries.txt'
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

class Parser:
    def __init__(self):
        pass
    
    @classmethod
    def remove_comments(cls,s):
        s = s[:s.find('//')].strip() #Remove comments
        return s
    @classmethod
    def splitlr(cls,s):
        left,right = s.split(':=')
        left = left.strip()
        right = right.strip()
        return left,right

    @classmethod
    def remove_outer_paren(cls,s):
        s = s[s.find('(')+1:s.find(')')]
        return s
    
    @classmethod
    def parse(cls,s,type):
        if type == 'input':
            return cls.remove_outer_paren(s)
        elif type == 'select':
            s = s[s.find("(")+1:s.rfind(")")]
            base_table,args = s.split(',')
            base_table = base_table.strip()
            args = args.strip()
            
            if 'or' not in args and 'and' not in args:
                return base_table,args,None
            condition = 'or' if 'or' in args else 'and'
            
            args = args.split(condition)
            args = [cls.remove_outer_paren(s.strip()) for s in args]
            return base_table,args,condition
        elif type == 'project':
            s = s[s.find("(")+1:s.rfind(")")]
            base_table,args = s.split(',')[0],s.split(',')[1:]
            base_table = base_table.strip()
            args = [a.strip() for a in args]
            
            return base_table,args
        
        elif type == 'avg':
            s = s[s.find("(")+1:s.rfind(")")]
            base_table,args = s.split(',')[0],s.split(',')[1]
            base_table = base_table.strip()
            args = args.strip()
            return base_table,args
        
    @classmethod
    def npy_to_dict(cls,arr):
        D = AttrDict()
        r,c = arr.shape
        keys = arr[0]
        for i,k in enumerate(keys):
            try:
                D[k] = arr[1:,i].astype(int)
            except:
                D[k] = arr[1:,i]
        return D

class Table:
    def __init__(self,table_name,inp_name = None, t = None):
        self.name = table_name
        if inp_name:
            arr = np.loadtxt(f"{inp_name}.txt",dtype=str,delimiter = "|")
            self.t = Parser.npy_to_dict(arr)
            print(f'{inp_name} is loaded and stored in {self.name}')
        else:
            self.t = t

    def select(self,args,condition,name):
        if condition:
            args = ['self.t.' + arg for arg in args]
            indices = []
            for a in args:
                i = eval(f'np.where({a})')
                indices.append(i)
                
            if condition == 'or':
                indices = reduce(np.union1d, indices)
            else:
                indices = reduce(np.intersect1d, indices)
            
        else:
            args = args.replace('=','==')
            args = 'self.t.' + args
            indices = eval(f'np.where({args})')

        new_table = {k:v[indices] for k,v in self.t.items()}
        new_table = AttrDict(new_table)
        
        new_table = Table(name, t = new_table)
        return new_table
    
    def project(self,args,name):
        new_table = {k:self.t[k] for k in args}
        new_table = AttrDict(new_table)
        
        new_table = Table(name, t = new_table)
        return new_table
        
    def avg(self,args):
        
        arr = self.t[args]
        ans = np.mean(arr)
        print(f'AVG({args}) = {ans}')
        return ans
        
    def __str__(self):
        keys = list(self.t.keys())
        n = len(self.t[keys[0]])
        s = '|'.join(keys) + '\n'
        for i in range(n):
            s += '|'.join([str(self.t[k][i]) for k in keys])
            s += '\n'
                
        return s
    def __repr__(self):
        return self.__str__()

class Solution:
    def __init__(self):
        pass
    def read_line(self,line):
        line = Parser.remove_comments(line)
        if line:
            if ':=' in line:
                left,right = Parser.splitlr(line)
                if 'inputfromfile' in right:
                    inp_name = Parser.parse(right,'input')
                    exec(f'self.{left} = Table(left,inp_name)')
                    
                elif 'select' in right:
                    base_table, args,condition = Parser.parse(right,'select')
                    exec(f'self.{left} = self.{base_table}.select(args,condition,left)')
                    
                elif 'project' in right:
                    base_table, args = Parser.parse(right,'project')
                    exec(f'self.{left} = self.{base_table}.project(args,left)')
                    
                elif 'avggroup' in right:
                    pass
                elif 'movavg' in right:
                    pass
                
                elif 'avg' in right:
                    base_table, args = Parser.parse(right,'avg')
                    exec(f'self.{left} = self.{base_table}.avg(args)')
                
                    
    

In [41]:
solution = Solution()
with open(query_file,'r') as f:
    for i,line in enumerate(f):
        
        solution.read_line(line)
# 
                    

sales1 is loaded and stored in R
AVG(qty) = 23.736666666666668
sales2 is loaded and stored in S


In [37]:
line



'T3 := movavg(T2prime, R1_qty, 3) // perform the three item moving average of T2prime\n'